# 我的赛程总结

# 冠军方案
https://mp.weixin.qq.com/s/-lizDyP2y357plcG1M64TA

代码地址：https://github.com/guoday/Tencent2020_Rank1st

从初赛冠军、复赛冠军，然后到最佳答辩，一路披荆斩棘，再次感谢队友（中山大学微软亚洲研究院联合培养博士生郭达雅、多次荣获国内NLP和数据挖掘比赛top名次选手梁少强，赛圈id“大白”）。

## 特征工程
### 统计特征

    用户出现的总次数和天数
    用户点击广告的总次数
    用户点击不同广告、产品、类别、素材、广告主的总数
    用户每天每条广告点击的平均次数，均值和方差
### 概率分布特征
![](img/tx01.png)


## 模型介绍
先来看下模型框架部分，主要分为三个部分。
### 整体
![](img/tx02.png)

### bert
![](img/tx03.png)
首先是bert部分，假设每个广告为一个单词，将用户点击序列看作一句话，这样就将问题转为了NLP的文本分类问题。我们使用bert作为我们的encoder进行编码，这里我们使用了12层transformer。每层transoformer将上一层的隐变量作为输入，经过Multi-Head Self Attention和Layer Norm等，最后输出交互更高的隐变量。

我们知道bert在NLP取得巨大成功的原因主要是预训练，所以在比赛初期不断尝试将预训练运用到本赛题中。但与NLP不同，广告具有稀疏性，使其词表达到七八百万，这里会造成两个问题。第一个问题是这么大的词表很难放入GPU中，第二个问题是即使放到CPU，由于参数过大，也难于训练。针对这些问题，我们使用了两阶段的预训练方法对BERT进行训练。

#### 预训练—Word2Vector
第一阶段是使用word2vec去预训练广告的embedding，目标是将类似的广告进行聚类，具体操作如下图所示，将用户点击广告序列输入到embedding层，然后预测广告id。但这里会存在两个问题，首先是我们只使用了一个广告去预测另一个广告，并没有使用更多的广告进行预测，因此缺失了更多的上下文信息。其次是没有利用广告本身的属性进行预测，比如类别信息，或者年龄性别信息。比如给定该广告是一个游戏类型的广告，那么就更容易预测出当前广告，同时也鼓励模型结合最近点击的广告和当前广告的属性进行预测，从而对齐广告id及其属性的embedding。
![](img/tx04.png)

#### 预训练—Masked Language Modeling (MLM)
鉴于以上问题，我们对Masked Language Modeling（MLM）进行改进，原始的MLM是直接mask掉整个广告包括属性，但我们只mask广告的一部分（广告id或者属性）。

具体地，我们随机mask掉20%的token，mask后的位置使用0向量来代替，或者采样一个属性id来替换。使用改进的MLM预测这些广告的时候，不仅可以使用最近点击的广告，也可以用它的属性去预测，使得不同属性的embedding能够在语义空间上对齐。同时，我们也采用了一个很关键的策略，我们把词表缩小到10万（提取top10w，其余为unk），这样使得bert能跑起来。对于原始的MLM，我们也尝试过，虽然有提高但提高不是很大。
![](img/tx05.png)

最后预测这些id的时候，不仅可以使用之前之后点击的广告，也可以用它的属性去预测。

### 融合层—Fusion Layer
![](img/tx06.png)
把点击广告的受众人群性别年龄分布融入到网络之中，具体的做法是把人群分为五个部分，假设用户点击n个广告并来自第一个人群，那么可以把剩下的作为种子人群，计算出每个广告的性别年龄概率分布，用四层的transformer进行融合。
### 输出层
![](img/tx07.png)


## 思考与总结
### 主要创新

#### 改进BERT并运用到人口属性预测场景

分阶段预训练，缓解广告稀疏性问题并加快预训练速度 （4*V100 预训练12个小时）

改进MLM预训练目标，并从多维度学习广告及其属性的语义表示

将BERT运用到人口属性预测的场景，从性能上验证了预训练在广告领域的潜力

#### 提出融合后验概率分布的方法及模型

利用受众人群求出每个广告的概率分布

利用多层Transformer融合概率分布及BERT的语义表示，能有效提升性能
### 问题思考

#### 预训练模型越大越好？
![](img/tx08.png)

#### 如何进一步改善预训练模型？

预训练目标
支持图输入的预训练模型
广告稀疏性问题，例如在未脱敏的情况下使用BPE算法
